In [1]:
import numpy as np
import math
from functools import reduce
import time
import sys
import os
import numpy as np

path_project = "\\".join(os.getcwd().split("\\")[:-1])
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, path_project)
from pathlib import Path
import nidaqmx

def lcm(a, b):
    """Calculate the least common multiple of two numbers.""" 
    return abs(a * b) // math.gcd(a, b)


def lcm_of_list(numbers):
    """Find the LCM of a list of numbers."""
    return reduce(lcm, numbers)

def seqtime(seq_tb):
    return np.sum([pulse[-1] for pulse in seq_tb])

# some constants
Hz = 1e-9 # GHz
kHz = 1e-6 # GHz
MHz = 1e-3 # GHz
pi = np.pi


In [2]:
import nidaqmx
from nidaqmx.constants import TerminalConfiguration, VoltageUnits, Edge, AcquisitionType, READ_ALL_AVAILABLE
from nidaqmx.stream_readers import AnalogSingleChannelReader

from hardware import config as hcf
from hardware.hardwaremanager import HardwareManager
from hardware.pulser.pulser import (
    OutputState,
    TriggerStart,
    TriggerRearm,
    HIGH,
    LOW,
    INF,
    REPEAT_INFINITELY
)
timebase = lcm_of_list(
    [hcf.VDISYN_timebase, hcf.SIDIG_timebase, hcf.PS_timebase, hcf.RSRF_timebase]
)

hm = HardwareManager()
# adds synthesizer, laser, and pulse generator 
hm.add_default_hardware()


Connect to Pulse Streamer via JSON-RPC.
IP / Hostname: 169.254.8.2
Pulse Streamer 8/2 firmware: v1.7.2
Client software: v1.7.0
VDI Sythesizer Serail Port Open


In [3]:
hm.windfreak.set_output(channel=0, freq=600e6, power=0, phase=0.0)
hm.windfreak.set_output(channel=1, freq=600e6, power=0, phase=0.0)

In [5]:
# 10% duty cycle 
T = 100e3  # µs per cycle
on_rf = 10e3  # µs rfA on duration
pre_blk = 1e3  # µs blk before rfA
post_blk = 1e3  # µs blk after rfA
idle = T - (pre_blk + on_rf + post_blk)
one_cycle = [
    (["BLK"], pre_blk),              # blk on before rfA
    (["BLK", "rfA"], on_rf),    
    (["rfB"],on_rf),     # blk + rfA
    (["BLK"], post_blk),            # blk on after rfA
    ([], idle)                      # idle
]
sequence = one_cycle   # repeat to reach 1s

In [ ]:
# # 20% duty cycle 
# T = 100e3  # µs per cycle
# on_rf = 20e3  # µs rfA on duration
# pre_blk = 1e3  # µs blk before rfA
# post_blk = 1e3  # µs blk after rfA
# idle = T - (pre_blk + on_rf + post_blk)
# one_cycle = [
#     (["BLK"], pre_blk),              # blk on before rfA
#     (["BLK", "rfA"], on_rf),         # blk + rfA
#     (["BLK"], post_blk),            # blk on after rfA
#     ([], idle)                      # idle
# ]
# sequence = one_cycle   # repeat to reach 1s

In [6]:
hm.pg.setSequence(sequence, reset=True)

hm.pg.setTrigger(start=TriggerStart.SOFTWARE, rearm=TriggerRearm.MANUAL)
hm.pg.plotSeq(plot_all=False)


In [7]:
hm.pg.stream(n_runs=10000)

for i in range(100000000):    
    hm.pg.startNow()
    if hm.pg.hasFinished():
        time.sleep(1)
        hm.pg.rearm()


KeyboardInterrupt: 

In [8]:
hm.pg.reset()

ReadTimeout: HTTPConnectionPool(host='169.254.8.2', port=8050): Read timed out. (read timeout=10)

In [ ]:
hm.pg.forceFinal()
